# Módulo 5 Tarefa 1 - Vivian Sant Anna
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  OK

2. Conte o número de valores *missing* por variável. OK 

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  OK

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  OK

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br> OK
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  OK
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [3]:
# 2) seu código aqui
# Carregando o arquivo em um DataFrame
sinasc_real = pd.read_csv('SINASC_RO_2019.csv')

# Contando valores missing no DataFrame real
missing_values_real = sinasc_real.isnull().sum()
missing_values_real

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [4]:
# 3) seu código aqui
# Selecionando as colunas especificadas do DataFrame
colunas_selecionadas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                        'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']

sinasc_selecionado = sinasc_real[colunas_selecionadas]

sinasc_selecionado.head()  # Mostrando as primeiras linhas do DataFrame selecionado para verificação


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [5]:
#Encontrando missings
# Contando valores missing no DataFrame selecionado
missing_values_selecionado = sinasc_selecionado.isnull().sum()
missing_values_selecionado


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [6]:
# 4) seu código aqui
# Removendo registros onde 'APGAR5' não está preenchido
sinasc_selecionado_limpo = sinasc_selecionado.dropna(subset=['APGAR5'])

# Contando o número de linhas após a remoção
num_linhas = sinasc_selecionado_limpo.shape[0]

# Contando valores missing no DataFrame após a remoção
missing_values_apos_limpeza = sinasc_selecionado_limpo.isnull().sum()

num_linhas, missing_values_apos_limpeza


(26925,
 LOCNASC          0
 IDADEMAE         0
 ESTCIVMAE      315
 ESCMAE         310
 QTDFILVIVO    1566
 GESTACAO      1216
 GRAVIDEZ        76
 CONSULTAS        0
 APGAR5           0
 dtype: int64)

In [8]:
# 5) seu código aqui
# Substituindo valores não preenchidos (NaN) por 9 nas colunas 'ESTCIVMAE' e 'CONSULTAS'
sinasc_selecionado_limpo['ESTCIVMAE'].fillna(9, inplace=True)
sinasc_selecionado_limpo['CONSULTAS'].fillna(9, inplace=True)

# Contando novamente valores missing no DataFrame após a substituição
missing_values_apos_substituicao = sinasc_selecionado_limpo.isnull().sum()
missing_values_apos_substituicao


/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/3059101275.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['ESTCIVMAE'].fillna(9, inplace=True)
/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/3059101275.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['CONSULTAS'].fillna(9, inplace=True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [9]:
# 6) Seu código aqui
# Substituindo valores não preenchidos (NaN) por 0 na coluna 'QTDFILVIVO'
sinasc_selecionado_limpo['QTDFILVIVO'].fillna(0, inplace=True)

# Contando novamente valores missing no DataFrame após a substituição
missing_values_apos_substituicao_qtdfilvivo = sinasc_selecionado_limpo.isnull().sum()
missing_values_apos_substituicao_qtdfilvivo


/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/3631374774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['QTDFILVIVO'].fillna(0, inplace=True)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [10]:
# 7) seu código aqui
# Preenchendo valores ausentes nas colunas restantes

# ESCMAE: Preenchido com 'Desconhecido' para evitar suposições sobre a variabilidade da escolaridade.
sinasc_selecionado_limpo['ESCMAE'].fillna('Desconhecido', inplace=True)

# GESTACAO: Preenchido com '37 a 41 semanas', baseado na duração média comum da gestação humana.
sinasc_selecionado_limpo['GESTACAO'].fillna('37 a 41 semanas', inplace=True)

# GRAVIDEZ: Preenchido com 'Desconhecido' devido à variabilidade e especificidade do tipo de gravidez.
sinasc_selecionado_limpo['GRAVIDEZ'].fillna('Desconhecido', inplace=True)

# Verificando novamente valores missing no DataFrame após a substituição
missing_values_final = sinasc_selecionado_limpo.isnull().sum()


/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/1079394123.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['ESCMAE'].fillna('Desconhecido', inplace=True)
/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/1079394123.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['GESTACAO'].fillna('37 a 41 semanas', inplace=True)
/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/1079394123.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [11]:
# 8) seu código aqui
# Categorização da variável APGAR5 de acordo com as regras fornecidas
def categoriza_apgar(valor):
    if valor >= 8:
        return 'Normal'
    elif valor >= 6:
        return 'Asfixia Leve'
    elif valor >= 4:
        return 'Asfixia Moderada'
    else:
        return 'Asfixia Severa'

sinasc_selecionado_limpo['APGAR5_CAT'] = sinasc_selecionado_limpo['APGAR5'].apply(categoriza_apgar)

# Calculando as frequências para cada categoria
frequencias_apgar = sinasc_selecionado_limpo['APGAR5_CAT'].value_counts()
frequencias_apgar


/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/138633002.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo['APGAR5_CAT'] = sinasc_selecionado_limpo['APGAR5'].apply(categoriza_apgar)


APGAR5_CAT
Normal              26463
Asfixia Leve          320
Asfixia Severa         74
Asfixia Moderada       68
Name: count, dtype: int64

In [12]:
# 9) seu código aqui
# Renomeando as colunas para o formato snake case
sinasc_selecionado_limpo.rename(columns={
    'LOCNASC': 'locnasc',
    'IDADEMAE': 'idademae',
    'ESTCIVMAE': 'estcivmae',
    'ESCMAE': 'escmae',
    'QTDFILVIVO': 'qtdfilvivo',
    'GESTACAO': 'gestacao',
    'GRAVIDEZ': 'gravidez',
    'CONSULTAS': 'consultas',
    'APGAR5': 'apgar5',
    'APGAR5_CAT': 'apgar5_cat'
}, inplace=True)

# Verificando o cabeçalho do DataFrame para confirmar as mudanças
sinasc_selecionado_limpo.head()


/var/folders/__/922ks3293zn8ylx07ry7lpfc0000gn/T/ipykernel_7237/58470578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sinasc_selecionado_limpo.rename(columns={


,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
